### Лабораторна работа 3.
Выполнили студенты группы **6404**
- Бахаев Дмитрий В.
- Сагалов Никита Д.

**3.1 Написать функцию формирования порождающей и проверочной матриц кода Хэмминга (𝟐𝒓 − 𝟏, 𝟐𝒓 − 𝒓 − 𝟏, 𝟑) на основе параметра 𝒓, а также таблицы синдромов для всех однократных ошибок.**

In [11]:
import numpy as np
from itertools import combinations, product

class HammingCode:
    def __init__(self, r, mistakes=1):
        self.r = r
        self.H = self.__generate_checker_matrix()
        self.G = self.__generate_generator_matrix()
        self.mistakes = mistakes

    @property
    def mistakes(self):
        return self._mistakes
    
    @mistakes.setter
    def mistakes(self, value):
        # Перегенерируем таблицу синдромов только если поменялось число ошибок
        if not hasattr(self, "mistakes") or value != self._mistakes:
            self._mistakes = value
            self.syndrome_table = self.__generate_syndrome_table_n_mistakes(self._mistakes)

    # H матрица
    def __generate_checker_matrix(self):
        # Создаем пустую матрицу H размером (2^r - 1) строк и r столбцов
        H = np.zeros((2 ** self.r - 1, self.r), dtype=int)
        
        # Заполняем последние r строк единичной матрицей размером r на r
        H[-self.r:, :] = np.eye(self.r, dtype=int)

        # Генерируем все возможные двоичные комбинации длиной r
        binary_combinations = list(product([0, 1], repeat=self.r))
        
        # Заполняем строки матрицы комбинациями, где число единиц больше 1
        index = 0
        for combination in binary_combinations:
            if sum(combination) > 1:
                H[index, :] = combination
                index += 1

        return H
    
    # G матрица
    def __generate_generator_matrix(self):
        # Количество символов в коде Хэмминга
        n = 2 ** self.r - 1
        
        # Инициализация нулевой матрицы G размером (n - r) строк и n столбцов
        G = np.zeros((n - self.r, n), dtype=int)
        
        # Заполнение первых (n - r) столбцов единичной матрицей
        G[:, :n - self.r] = np.eye(n - self.r, dtype=int)
        
        # Копирование частей матрицы H для заполнения правой части матрицы G
        G[:, n - self.r:] = self.H[:n - self.r, :].copy()

        return G
    
    # Создание таблицы синдромов для n-кратных ошибок
    def __generate_syndrome_table_n_mistakes(self, n):
        syndrome_table = {}

        # Перебор всех возможных комбинаций позиций ошибок от 1 до n
        for num_errors in range(1, n+1):
            for error_positions in combinations(range(self.H.shape[0]), num_errors):
                word = np.zeros(self.H.shape[0], dtype=int)
                for pos in error_positions:
                    word[pos] = 1
                syndrome = word @ self.H % 2
                syndrome_table[tuple(syndrome)] = word

        return syndrome_table
    
    def test_n_mistakes(self, n=1):
        # Кодируем слово
        k_word = np.array([1] + [0] * (2 ** self.r - 2 - self.r))
        code_word = k_word @ self.G % 2

        # Вносим n-кратную ошибку
        error = np.array([1] * n + [0] * (self.G.shape[1] - n))
        received_word = (code_word + error) % 2

        # Вычисляем синдром и исправляем ошибку
        syndrome = received_word @ self.H % 2
        if tuple(syndrome) in self.syndrome_table:
            syndrome_error = self.syndrome_table[tuple(syndrome)]
            corrected_word = (received_word + syndrome_error) % 2
        else:
            corrected_word = received_word

        # Проверка
        print("Параметр r:                                  ", self.r)
        print("Начальное закодированное слово:              ", code_word)
        print("Ошибка:                                      ", error)
        print(f"Закодированное слово с {n} ошибками(ой):       ", received_word)
        print("Синдром:                                     ", syndrome)
        print("Ошибка исправлена:                           ", np.array_equal(code_word, corrected_word))

**3.2. Провести исследование кода Хэмминга для одно-, двух- и трёхкратных ошибок для 𝒓 = 𝟐, 𝟑, 𝟒.**

Функция для исследования реализована в классе HammingCode (test_n_mistakes)

In [ ]:
print("==============================")
hamming = HammingCode(r=2, mistakes=1)
hamming.test_n_mistakes(1)
print("------------------------------")
hamming.test_n_mistakes(2)
print("------------------------------")
hamming.test_n_mistakes(3)
print("==============================")
hamming = HammingCode(r=3, mistakes=1)
hamming.test_n_mistakes(1)
print("------------------------------")
hamming.test_n_mistakes(2)
print("------------------------------")
hamming.test_n_mistakes(3)
print("==============================")
hamming = HammingCode(r=4, mistakes=1)
hamming.test_n_mistakes(1)
print("------------------------------")
hamming.test_n_mistakes(2)
print("------------------------------")
hamming.test_n_mistakes(3)
print("==============================")